# CSE 253 — Task 2: Melody‑to‑Harmony with the Nottingham Dataset
This notebook clones the Nottingham ABC corpus, cleans and pre‑processes it, builds PyTorch dataloaders, trains a bidirectional LSTM with positional embeddings, and plots training / validation loss.

In [1]:
# -------------------- 0. Install & imports --------------------
from music21 import corpus, converter, chord, note, meter, stream
from collections import defaultdict, Counter
import random, numpy as np
from tqdm import tqdm

random.seed(42)

# --------------------------------------------------------------
# Helper: return all elements at a given offset, handling BOTH
# music21 ≤ v8 (old keyword names) and ≥ v9 (new keyword names)
# --------------------------------------------------------------
def elements_at(stream_obj, offset, must_begin):
    """
    Parameters
    ----------
    stream_obj : music21.stream.Stream (usually .flat)
    offset      : float   – beat location you’re sampling
    must_begin  : bool    – True  → elements must *start* at offset
                             False → elements may simply overlap
    Returns
    -------
    list of elements (Notes, Chords, etc.) satisfying the criteria.
    Works no‑matter which version of music21 is installed.
    """
    try:  # music21 v9+ (new argument names)
        return stream_obj.getElementsByOffset(
            offset, offset,
            mustBeginInSpan=must_begin,
            includeEndBoundary=False,
            includeElementsThatEndAtStart=False
        )
    except TypeError:  # music21 v8‑ or older (legacy names)
        return stream_obj.getElementsByOffset(
            offset, offset,
            mustBeginInSpan=must_begin,
            includeEnd=False
        )


In [2]:
# -------------------- 1. Load Bach chorales --------------------
CHORALES = list(corpus.chorales.Iterator(numberingSystem='riemenschneider'))
print(f"{len(CHORALES)} chorales found.")   # ≈ 389

def quantize_parts(score, qlen=1.0):
    """
    Return (melody_pitches, chord_symbols) on a fixed quarter grid.
    Works across music21 v8 and v9.
    """
    soprano     = score.parts[0]
    chordified  = score.chordify()
    flat_melody = soprano.flat
    flat_chords = chordified.flat

    end = max(score.highestTime, chordified.highestTime)
    t, m_pitches, ch_syms = 0.0, [], []
    while t < end:

        # --- MELODY: any note that *overlaps* t  -----------------
        notes_here = [n for n in elements_at(flat_melody, t, must_begin=False)
                      if isinstance(n, note.Note)]
        if notes_here:
            # take highest pitch in case of voice‑leading overlaps
            m_pitches.append(max(notes_here, key=lambda n: n.pitch.midi).pitch.midi)
        else:
            m_pitches.append("rest")

        # --- CHORD: any sonority overlapping t  ------------------
        ch_here = [c for c in elements_at(flat_chords, t, must_begin=False)
                   if isinstance(c, chord.Chord)]
        if ch_here:
            c      = ch_here[0]
            symbol = f"{c.root().name}:{c.quality or 'maj'}"
        else:
            symbol = "N.C."
        ch_syms.append(symbol)

        t += qlen
    return m_pitches, ch_syms


371 chorales found.


In [3]:
# ---------- 3.  Split → train / valid ----------
data = [quantize_parts(s,qlen=0.5) for s in tqdm(CHORALES)]
# keep sequences with a sensible length
data = [(m, c) for m, c in data if len(m) > 16]
random.shuffle(data)
split = int(0.8*len(data))
train, valid = data[:split], data[split:]
print(f"Train {len(train)}  |  Valid {len(valid)}")

  0%|          | 0/371 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/music21/stream/base.py:4014: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  sIterator = self.iter().getElementsByOffset(
100%|██████████| 371/371 [00:16<00:00, 22.86it/s]

Train 296  |  Valid 75


In [4]:
# -------------------- 4. Beat‑aware frequency table --------------------
# Count occurrences:  P(chord | melody‑pitch‑class, beat‑index mod 4)
counts = defaultdict(Counter)
for mel, chords in train:
    for idx, (p, ch) in enumerate(zip(mel, chords)):
        if p == "rest" or ch == "N.C.":
            continue
        pc   = p % 12            # 0‑11
        beat = idx % 4           # assume 4‑beat bar
        counts[(pc, beat)][ch] += 1

# Deterministic mapping:   argmax chord for each (pc, beat)
mapping  = {key: cnts.most_common(1)[0][0] for key, cnts in counts.items()}
fallback = "C:maj"               # used when (pc, beat) never seen in training
print(f"Mapping built for {len(mapping)} (pitch‑class, beat) combos.")


Mapping built for 48 (pitch‑class, beat) combos.


In [5]:
# -------------------- 5. Define harmonize_melody and write_midi --------------------
def harmonize_melody(mel_line):
    harmony, last = [], fallback
    for idx, p in enumerate(mel_line):
        if p == "rest":
            harmony.append(last)          # sustain previous chord
            continue
        pc, beat = p % 12, idx % 4
        ch = mapping.get((pc, beat), fallback)
        harmony.append(ch)
        last = ch
    return harmony


def write_midi(mel, har, fp="bach_harmonisation_baseline.mid"):
    s = stream.Stream()
    part_mel, part_har = stream.Part(), stream.Part()
    for m_note, ch_sym in zip(mel, har):
        dur = 1.0
        # melody track
        n = note.Note(m_note) if m_note != "rest" else note.Rest()
        n.quarterLength = dur
        part_mel.append(n)
        # harmony track (simple root‑position triad)
        if ch_sym != "N.C.":
            root = note.Note(ch_sym.split(':')[0])
            tri  = chord.Chord([root,
                                root.transpose(4),
                                root.transpose(7)])
        else:
            tri = chord.Chord([])
        tri.quarterLength = dur
        part_har.append(tri)
    s.append([part_mel, part_har])
    s.write("midi", fp=fp)
    print("✔  MIDI written:", fp)

In [6]:
# ---------- 6. Evaluate & write a demo MIDI----------
hits, tot = 0, 0
for mel, gold_ch in valid:
    pred_ch = harmonize_melody(mel)
    for p, g in zip(pred_ch, gold_ch):
        if g != "N.C.":
            hits += (p == g)
            tot  += 1
print(f"Validation chord‑token accuracy: {hits/tot:.3f}")


mel_sample, _ = valid[0]
har_sample    = harmonize_melody(mel_sample)
write_midi(mel_sample, har_sample)


Validation chord‑token accuracy: 0.393
✔  MIDI written: bach_harmonisation_baseline.mid


In [7]:
# Count how many non‑rest melody tokens the chorale actually has
mel, gold = valid[0]
non_rest  = sum(1 for p in mel if p != "rest")
print(f"Total tokens in this melody: {len(mel)}")
print(f"Non‑rest tokens            : {non_rest}")


Total tokens in this melody: 96
Non‑rest tokens            : 45


In [8]:
# -------------------- 7.  Prepare vocabularies & integer‐encode sequences --------------------

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# 7.1. Collect all chord tokens and melody tokens from train/valid
all_chords   = set()
all_pitches  = set()

for mel, ch in train + valid:
    all_pitches.update([p for p in mel])     # p is int or "rest"
    all_chords.update([c for c in ch])       # c is chord‐string or "N.C."

# Build sorted vocab lists, including special PAD tokens
PITCH_TOKENS = sorted([p for p in all_pitches if p != "rest"])
PITCH_VOCAB  = ["<PAD>", "rest"] + [str(p) for p in PITCH_TOKENS]
pitch2idx    = {tok: idx for idx, tok in enumerate(PITCH_VOCAB)}
idx2pitch    = {idx: tok for tok, idx in pitch2idx.items()}

CHORD_VOCAB  = ["<PAD>", "N.C."] + sorted([c for c in all_chords if c != "N.C."])
chord2idx    = {tok: idx for idx, tok in enumerate(CHORD_VOCAB)}
idx2chord    = {idx: tok for tok, idx in chord2idx.items()}

print(f"‣  Melody‐vocab size = {len(PITCH_VOCAB)}  (incl. PAD, rest)")
print(f"‣  Chord‐vocab size  = {len(CHORD_VOCAB)}  (incl. PAD, N.C.)")

# 7.2. Helper to encode one (melody, chord) pair into integer sequences
def encode_sequence(mel_seq, chord_seq):
    """
    mel_seq   : list of length T, entries int or "rest"
    chord_seq : list of length T, entries chord‐string or "N.C."
    Returns:
      pitch_idx_seq : LongTensor[T]
      chord_idx_seq : LongTensor[T]
    """
    T = len(mel_seq)
    pitch_idxs = []
    chord_idxs = []
    for i in range(T):
        p = mel_seq[i]
        if p == "rest":
            pitch_idxs.append(pitch2idx["rest"])
        else:
            pitch_idxs.append(pitch2idx[str(p)])
        c = chord_seq[i]
        chord_idxs.append(chord2idx[c])
    return torch.LongTensor(pitch_idxs), torch.LongTensor(chord_idxs)

# 7.3. Encode all train/valid sequences
train_data = []
for mel, ch in train:
    pitch_idxs, chord_idxs = encode_sequence(mel, ch)
    train_data.append((pitch_idxs, chord_idxs, len(mel)))

valid_data = []
for mel, ch in valid:
    pitch_idxs, chord_idxs = encode_sequence(mel, ch)
    valid_data.append((pitch_idxs, chord_idxs, len(mel)))

print(f"‣  Number of training sequences: {len(train_data)}")
print(f"‣  Number of validation sequences: {len(valid_data)}")


‣  Melody‐vocab size = 29  (incl. PAD, rest)
‣  Chord‐vocab size  = 65  (incl. PAD, N.C.)
‣  Number of training sequences: 296
‣  Number of validation sequences: 75


In [9]:
# -------------------- 8.  PyTorch Dataset & DataLoader (with padding) --------------------

class ChoraleDataset(Dataset):
    def __init__(self, data_list):
        """
        data_list : list of tuples (pitch_idxs: LongTensor[T], chord_idxs: LongTensor[T], length: int)
        """
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pitch_idxs, chord_idxs, length = self.data[idx]
        return pitch_idxs, chord_idxs, length

    @staticmethod
    def collate_fn(batch):
        """
        batch: list of (pitch_idxs: LongTensor[T_i], chord_idxs: LongTensor[T_i], len_i)
        Returns:
          padded_pitches: (B, L) LongTensor
          padded_chords : (B, L) LongTensor
          lengths       : (B,) LongTensor
        """
        batch_size = len(batch)
        lengths = torch.LongTensor([item[2] for item in batch])
        max_len = lengths.max().item()

        padded_pitches = torch.full((batch_size, max_len),
                                    fill_value=pitch2idx["<PAD>"],
                                    dtype=torch.long)
        padded_chords = torch.full((batch_size, max_len),
                                   fill_value=chord2idx["<PAD>"],
                                   dtype=torch.long)

        for i, (p_seq, c_seq, L) in enumerate(batch):
            padded_pitches[i, :L] = p_seq
            padded_chords[i, :L] = c_seq

        return padded_pitches, padded_chords, lengths

# Hyperparameters for DataLoader
BATCH_SIZE = 32

train_dataset = ChoraleDataset(train_data)
valid_dataset = ChoraleDataset(valid_data)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=ChoraleDataset.collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=ChoraleDataset.collate_fn
)

# Inspect one batch
for batch in train_loader:
    p, c, L = batch
    print("pitches:", p.shape, "| chords:", c.shape, "| lengths:", L.shape)
    break


pitches: torch.Size([32, 136]) | chords: torch.Size([32, 136]) | lengths: torch.Size([32])


In [10]:
# -------------------- 9.  Bi-LSTM Model Definition --------------------

class BiLSTMChordTagger(nn.Module):
    def __init__(self,
                 n_pitch_tokens: int,
                 n_chord_tokens: int,
                 embed_dim: int = 128,
                 lstm_hidden: int = 256,
                 lstm_layers: int = 2,
                 dropout: float = 0.3):
        """
        n_pitch_tokens : size of pitch vocabulary (including PAD & 'rest')
        n_chord_tokens : size of chord vocabulary (including PAD & 'N.C.')
        """
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=n_pitch_tokens,
                                      embedding_dim=embed_dim,
                                      padding_idx=pitch2idx["<PAD>"])
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=lstm_hidden,
            num_layers=lstm_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if lstm_layers > 1 else 0.0
        )
        self.classifier = nn.Linear(in_features=lstm_hidden * 2,
                                    out_features=n_chord_tokens)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_pitches, lengths):
        """
        input_pitches : (B, L) LongTensor of pitch‐indices (padded with <PAD>)
        lengths       : (B,) LongTensor of actual lengths
        Returns:
          logits : (B, L, n_chord_tokens)
        """
        emb = self.embedding(input_pitches)  # (B, L, embed_dim)
        emb = self.dropout(emb)
        packed = nn.utils.rnn.pack_padded_sequence(emb,
                                                   lengths.cpu(),
                                                   batch_first=True,
                                                   enforce_sorted=False)
        packed_out, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        # out: (B, L, lstm_hidden * 2)
        logits = self.classifier(out)        # (B, L, n_chord_tokens)
        return logits

# Instantiate the model
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMChordTagger(
    n_pitch_tokens=len(PITCH_VOCAB),
    n_chord_tokens=len(CHORD_VOCAB),
    embed_dim=192,
    lstm_hidden=384,
    lstm_layers=3,
    dropout=0.4
).to(DEVICE)

print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")


BiLSTMChordTagger(
  (embedding): Embedding(29, 192, padding_idx=0)
  (lstm): LSTM(192, 384, num_layers=3, batch_first=True, dropout=0.4, bidirectional=True)
  (classifier): Linear(in_features=768, out_features=65, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)
Total parameters: 8921345


In [11]:
# -------------------- 10.  Loss, Optimizer, & Training Utilities --------------------

# Ignore "N.C." and "<PAD>" tokens during loss/accuracy
ignore_idx = chord2idx["N.C."]
pad_idx    = chord2idx["<PAD>"]

criterion = nn.CrossEntropyLoss(ignore_index=ignore_idx)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.5,
    patience=2
)

def compute_batch_accuracy(logits, targets):
    """
    logits  : (B, L, C) raw scores
    targets : (B, L) ground‐truth chord indices
    Returns:
      num_correct : int
      num_valid   : int
    """
    B, L, C = logits.shape
    flat_logits = logits.view(-1, C)        # (B*L, C)
    flat_preds  = flat_logits.argmax(dim=1)  # (B*L,)
    flat_targets= targets.view(-1)          # (B*L,)

    valid_mask = (flat_targets != ignore_idx) & (flat_targets != pad_idx)
    correct = (flat_preds == flat_targets) & valid_mask
    num_correct = correct.sum().item()
    num_valid   = valid_mask.sum().item()
    return num_correct, num_valid


In [12]:
# -------------------- 11.  Training & Validation Loop --------------------

NUM_EPOCHS    = 20
best_val_acc  = 0.0

train_losses  = []
valid_losses  = []
train_accs    = []
valid_accs    = []

for epoch in range(1, NUM_EPOCHS + 1):
    # ---- Training Phase ----
    model.train()
    epoch_loss    = 0.0
    epoch_correct = 0
    epoch_total   = 0

    for batch in train_loader:
        pitches, chords, lengths = batch
        pitches = pitches.to(DEVICE)
        chords  = chords.to(DEVICE)
        lengths = lengths.to(DEVICE)

        optimizer.zero_grad()
        logits = model(pitches, lengths)   # (B, L, n_chords)

        B, L, C = logits.shape
        flat_logits  = logits.view(B * L, C)
        flat_targets = chords.view(B * L)

        loss = criterion(flat_logits, flat_targets)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * B
        batch_correct, batch_total = compute_batch_accuracy(logits, chords)
        epoch_correct += batch_correct
        epoch_total   += batch_total

    train_loss = epoch_loss / len(train_dataset)
    train_acc  = epoch_correct / epoch_total

    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # ---- Validation Phase ----
    model.eval()
    val_loss    = 0.0
    val_correct = 0
    val_total   = 0

    with torch.no_grad():
        for batch in valid_loader:
            pitches, chords, lengths = batch
            pitches = pitches.to(DEVICE)
            chords  = chords.to(DEVICE)
            lengths = lengths.to(DEVICE)

            logits = model(pitches, lengths)  # (B, L, n_chords)

            B, L, C = logits.shape
            flat_logits  = logits.view(B * L, C)
            flat_targets = chords.view(B * L)

            loss = criterion(flat_logits, flat_targets)
            val_loss += loss.item() * B

            batch_correct, batch_total = compute_batch_accuracy(logits, chords)
            val_correct += batch_correct
            val_total   += batch_total

    val_loss = val_loss / len(valid_dataset)
    val_acc  = val_correct / val_total

    valid_losses.append(val_loss)
    valid_accs.append(val_acc)

    scheduler.step(val_acc)

    print(f"Epoch {epoch:02d}  | "
          f"Train Loss = {train_loss:.4f}  Acc = {train_acc:.4f}  | "
          f"Val Loss = {val_loss:.4f}  Acc = {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_bilstm_chord_tagger.pt")

print(f"\n→  Best validation chord‐token accuracy = {best_val_acc:.4f}")


Epoch 01  | Train Loss = 3.9957  Acc = 0.1402  | Val Loss = 3.9217  Acc = 0.1650
Epoch 02  | Train Loss = 3.9067  Acc = 0.2244  | Val Loss = 3.7946  Acc = 0.3045
Epoch 03  | Train Loss = 3.8232  Acc = 0.3597  | Val Loss = 3.7052  Acc = 0.3528
Epoch 04  | Train Loss = 3.7239  Acc = 0.4162  | Val Loss = 3.6122  Acc = 0.4257
Epoch 05  | Train Loss = 3.6821  Acc = 0.4429  | Val Loss = 3.5735  Acc = 0.4617
Epoch 06  | Train Loss = 3.6711  Acc = 0.4829  | Val Loss = 3.5329  Acc = 0.4809
Epoch 07  | Train Loss = 3.5847  Acc = 0.5170  | Val Loss = 3.4812  Acc = 0.5091
Epoch 08  | Train Loss = 3.5848  Acc = 0.5330  | Val Loss = 3.4561  Acc = 0.5114
Epoch 09  | Train Loss = 3.5025  Acc = 0.5554  | Val Loss = 3.4208  Acc = 0.5419
Epoch 10  | Train Loss = 3.5341  Acc = 0.5735  | Val Loss = 3.4007  Acc = 0.5593
Epoch 11  | Train Loss = 3.4621  Acc = 0.5911  | Val Loss = 3.3664  Acc = 0.5629
Epoch 12  | Train Loss = 3.4173  Acc = 0.6081  | Val Loss = 3.3492  Acc = 0.5611
Epoch 13  | Train Loss = 3.4

In [13]:
# -------------------- 13.  Evaluate on Validation & Demo MIDI --------------------

# Reload best model
model.load_state_dict(torch.load("best_bilstm_chord_tagger.pt"))
model.to(DEVICE)
model.eval()

# Compute final validation accuracy
val_correct = 0
val_total   = 0

with torch.no_grad():
    for batch in valid_loader:
        pitches, chords, lengths = batch
        pitches = pitches.to(DEVICE)
        chords  = chords.to(DEVICE)
        lengths = lengths.to(DEVICE)

        logits = model(pitches, lengths)  # (B, L, C)
        batch_correct, batch_total = compute_batch_accuracy(logits, chords)
        val_correct += batch_correct
        val_total   += batch_total

final_val_acc = val_correct / val_total
print(f"✔  Final validation chord‐token accuracy = {final_val_acc:.4f}")

# 13.1. Generate & write one demo MIDI using the trained model
def predict_chords_from_melody(mel_seq):
    """
    Given a melody sequence mel_seq (list of int or "rest"),
    return a list of predicted chord‐strings.
    """
    model.eval()
    pitch_idxs, _ = encode_sequence(mel_seq, ["N.C."] * len(mel_seq))
    L = len(mel_seq)
    pitch_idxs = pitch_idxs.unsqueeze(0).to(DEVICE)        # (1, L)
    lengths = torch.LongTensor([L]).to(DEVICE)

    with torch.no_grad():
        logits = model(pitch_idxs, lengths)                 # (1, L, C)
        preds  = logits.argmax(dim=2).squeeze(0).cpu().tolist()

    pred_chord_seq = [idx2chord[p] for p in preds]
    return pred_chord_seq

# Pick the first validation example and write a demo MIDI
mel_sample, gold_chords = valid[0]
pred_chords = predict_chords_from_melody(mel_sample)

print(f"Example  →  Melody length = {len(mel_sample)} tokens")
print(f"Ground‐truth chords (first 16): {gold_chords[:16]}")
print(f"Predicted chords    (first 16): {pred_chords[:16]}")

write_midi(mel_sample, pred_chords, fp="demo_bilstm_harmonisation.mid")


✔  Final validation chord‐token accuracy = 0.6098
Example  →  Melody length = 96 tokens
Ground‐truth chords (first 16): ['N.C.', 'N.C.', 'N.C.', 'A:minor', 'N.C.', 'E:major', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'N.C.', 'A:minor']
Predicted chords    (first 16): ['E:major', 'E:major', 'E:major', 'A:minor', 'A:minor', 'E:major', 'E:major', 'A:minor', 'A:minor', 'D:minor', 'E:major', 'A:minor', 'E:major', 'E:major', 'A:minor', 'A:minor']
✔  MIDI written: demo_bilstm_harmonisation.mid
